# LDA Model

>Now that the raw input text has been tokenized/cleaned, we can train our LDA model.
>
>The below will train our model and then save it for later usage.
>
>Some *seed params* have been hardcoded to nudge the model in the right direction.

## Read Text Input

In [1]:
from gensim.corpora.dictionary import Dictionary
from lda_helpers import read_lda_input  # Package with helpers

texts = read_lda_input('lda_input/lda_input.jl')
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

## Construct Eta Prior Matrix

>Allowing the model to train randomly will result in not-so-understandable genres.
>
>Thus we provide some seed words to push the model towards some desired output genres.
>
>Construct the corresponding matrix to pass as the *eta* parameter.
>
>This serves as a prior assumption of each topic distribution from which we initialize our LDA model.

In [2]:
import numpy as np
from lda_input.lda_seed import genre_seed_words, r, smooth  # Hardcoded params used to train LDA model

seed_structs = [dict.fromkeys(x[1], smooth/len(x[1])) for x in genre_seed_words]

k = len(genre_seed_words)  # Topic Count
n = len(id2word)           # Vocabulary size

# Convert to prior eta matrix, for LDA starting point
f = lambda i,j: seed_structs[i].get(id2word[j], (1-smooth)/(n-len(seed_structs[i])))
seed_matrix = np.fromfunction(np.vectorize(f, otypes=[np.float64]), (k,n), dtype=int)

## Train LDA Model

In [3]:
from lda_helpers import get_lda_model
import warnings
warnings.filterwarnings('ignore')  # Annoying RunTimeError which doesn't affect anything

lda_model = get_lda_model(corpus, id2word, k, r=r, eta=seed_matrix)

## Visualize LDA Genres

In [4]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
LDAvis_display = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, sort_topics=False)
LDAvis_display

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.131275  0.054746       1        1  22.352630
1      0.079864 -0.320354       2        1  13.224697
2     -0.234813  0.091432       3        1  18.041046
3     -0.008951  0.085365       4        1  14.246521
4      0.327067  0.161125       5        1   7.794986
5     -0.031893 -0.072314       6        1  24.340120, topic_info=        Term         Freq        Total Category  logprob  loglift
255     life  2959.000000  2959.000000  Default  30.0000  30.0000
383   school  2779.000000  2779.000000  Default  29.0000  29.0000
148     team   944.000000   944.000000  Default  28.0000  28.0000
1350    club   856.000000   856.000000  Default  27.0000  27.0000
899   family   998.000000   998.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
80     young   365.340585  1194.052506   Topic6  -4.9600   0.2288
154    world   428.968671  2465.412900   Topic6  -4.7994  -0.3357
255     life   337.577138  2959.063043   Topic6  -5.0390  -0.7578
406    begin   284.349168  1361.010241   Topic6  -5.2106  -0.1527
940     best   252.724214   605.221521   Topic6  -5.3285   0.5397

[542 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
656       1  0.157006  ability
656       2  0.001892  ability
656       3  0.459668  ability
656       4  0.088907  ability
656       5  0.009458  ability
...     ...       ...      ...
80        4  0.071186    young
80        5  0.055274    young
80        6  0.305682    young
762       5  0.997847     zero
2918      2  1.003642   zombie

[927 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6])

## Save Model to Disk

>Use gensim's *save* functionality to store this model for later usage.

In [5]:
from os import mkdir
try:
    mkdir('lda_model')
except FileExistsError:
    pass
lda_model.save('lda_model/lda_model')

C:\Users\Mihir\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
